# Метод опорных векторов

**Метод опорных векторов** (*Support Vector Machine, SVM*) - один из видов линейных классификаторов. Функционал, который он оптимизирует, направлен на максимизацию ширины разделяющей полосы между классами.

В случае линейно разделимой выборки нужен для максимизации ширины между классами.      
В случае линейно неразделимой выборки просто добавляются штрафы за попадания объекта за полосу.

In [34]:
import pandas as pd
from sklearn.svm import SVC
import numpy as np

In [35]:
data = pd.read_csv('svm-data.csv', header=None)

In [36]:
data.head()

,0,1,2
0,0.0,0.70,0.29
1,1.0,0.23,0.55
2,0.0,0.72,0.42
3,0.0,0.98,0.68
4,0.0,0.48,0.39


In [37]:
# Классификатор - линейный --> ядро - linear
y = data[0]
X = data.loc[:, 1:]
clf = SVC(kernel='linear', C=100000, random_state=241)
clf.fit(X, y)

SVC(C=100000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

In [38]:
# индексы опорных объектов
print(clf.support_)

[3 4 9]
